In [ ]:
# CELL 1: Install dependencies
import sys, subprocess
pkgs = ["pandas", "numpy", "plotly", "ipywidgets", "kaleido"]
subprocess.check_call([sys.executable, "-m", "pip", "install"] + pkgs + ["--quiet"])
print("Done!")

In [ ]:
# CELL 2: Imports
import os
import re
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

print("="*60)
print("🏆 ML Model Comparison Dashboard")
print("Compare multiple ML models and analyze feature importance")
print("="*60)

In [ ]:
# CELL 3: Folder Browser

class FolderBrowser:
    """Interactive folder browser widget."""
    
    def __init__(self, start='.', label='Folder'):
        self.cur = Path(start).resolve()
        self.sel = self.cur
        self.label = label
        
        self.html = widgets.HTML(f"<code>{self.cur}</code>")
        self.dd = widgets.Select(
            options=self._list(), 
            layout=widgets.Layout(width='100%', height='150px')
        )
        self.b_up = widgets.Button(description='↑ Up', layout=widgets.Layout(width='70px'))
        self.b_in = widgets.Button(description='→ Enter', layout=widgets.Layout(width='80px'))
        self.b_sel = widgets.Button(description='✓ Select', button_style='success', 
                                    layout=widgets.Layout(width='80px'))
        self.selhtml = widgets.HTML(f"<b>Selected:</b> <code>{self.sel}</code>")
        
        self.b_up.on_click(lambda b: self._up())
        self.b_in.on_click(lambda b: self._enter())
        self.b_sel.on_click(lambda b: self._select())
        
        self.w = widgets.VBox([
            widgets.HTML(f"<b>{label}</b>"),
            self.html, self.dd,
            widgets.HBox([self.b_up, self.b_in, self.b_sel]),
            self.selhtml
        ])
    
    def _list(self):
        try:
            items = ['.']
            for x in sorted(self.cur.iterdir()):
                if x.is_dir() and not x.name.startswith('.'):
                    items.append(f"📁 {x.name}")
            return items
        except:
            return ['.']
    
    def _refresh(self):
        self.html.value = f"<code>{self.cur}</code>"
        self.dd.options = self._list()
    
    def _up(self):
        if self.cur.parent != self.cur:
            self.cur = self.cur.parent
            self._refresh()
    
    def _enter(self):
        if self.dd.value and self.dd.value.startswith('📁'):
            folder_name = self.dd.value.replace('📁 ', '')
            p = self.cur / folder_name
            if p.is_dir():
                self.cur = p
                self._refresh()
    
    def _select(self):
        self.sel = self.cur
        self.selhtml.value = f"<b>Selected:</b> <code>{self.sel}</code>"
    
    def path(self):
        return self.sel

print("Folder browser defined.")

In [ ]:
# CELL 4: Results Scanner Class

class MLResultsScanner:
    """Scan and aggregate ML results from multiple model folders."""
    
    def __init__(self):
        self.results_dir = None
        self.models = {}  # {model_name: {metrics: df, importance: df, ...}}
        self.metrics_df = None  # Combined metrics
        self.importance_df = None  # Combined importance
    
    def scan(self, root_path: Path) -> Dict:
        """Scan directory for ML results."""
        self.results_dir = Path(root_path)
        self.models = {}
        
        # Find all subdirectories with results
        for subdir in self.results_dir.iterdir():
            if not subdir.is_dir() or subdir.name.startswith('.'):
                continue
            
            # Look for metrics and importance files
            metrics_files = list(subdir.glob('*_metrics_*.csv'))
            importance_files = list(subdir.glob('*_feature_importance_*.csv'))
            summary_files = list(subdir.glob('*_summary_*.txt'))
            prediction_files = list(subdir.glob('*_predictions_*.csv'))
            
            if metrics_files:
                model_name = subdir.name
                self.models[model_name] = {
                    'path': subdir,
                    'metrics_file': metrics_files[0] if metrics_files else None,
                    'importance_file': importance_files[0] if importance_files else None,
                    'summary_file': summary_files[0] if summary_files else None,
                    'prediction_file': prediction_files[0] if prediction_files else None,
                }
        
        return {
            'n_models': len(self.models),
            'models': list(self.models.keys())
        }
    
    def load_all(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Load all metrics and importance data."""
        metrics_list = []
        importance_list = []
        
        for model_name, info in self.models.items():
            # Load metrics
            if info['metrics_file']:
                try:
                    df = pd.read_csv(info['metrics_file'])
                    df['model_folder'] = model_name
                    metrics_list.append(df)
                except Exception as e:
                    print(f"Error loading metrics for {model_name}: {e}")
            
            # Load importance
            if info['importance_file']:
                try:
                    df = pd.read_csv(info['importance_file'])
                    df['model_folder'] = model_name
                    importance_list.append(df)
                except Exception as e:
                    print(f"Error loading importance for {model_name}: {e}")
        
        # Combine
        self.metrics_df = pd.concat(metrics_list, ignore_index=True) if metrics_list else None
        self.importance_df = pd.concat(importance_list, ignore_index=True) if importance_list else None
        
        return self.metrics_df, self.importance_df
    
    def get_model_comparison(self) -> pd.DataFrame:
        """Create model comparison table."""
        if self.metrics_df is None:
            return None
        
        # Pivot to get metrics by model
        # Filter for test split only
        test_metrics = self.metrics_df[self.metrics_df['split'] == 'test'].copy()
        
        # Pivot
        comparison = test_metrics.pivot_table(
            index='model_folder',
            columns='metric',
            values='value',
            aggfunc='first'
        ).reset_index()
        
        # Rename columns for clarity
        comparison.columns.name = None
        
        # Sort by R² descending
        r2_col = [c for c in comparison.columns if 'R' in c and '2' in c or 'R²' in c]
        if r2_col:
            comparison = comparison.sort_values(r2_col[0], ascending=False)
        
        return comparison
    
    def get_weighted_importance(self) -> pd.DataFrame:
        """Calculate feature importance weighted by model R²."""
        if self.importance_df is None or self.metrics_df is None:
            return None
        
        # Get R² for each model (test set)
        r2_by_model = {}
        for model in self.models.keys():
            model_metrics = self.metrics_df[
                (self.metrics_df['model_folder'] == model) & 
                (self.metrics_df['split'] == 'test')
            ]
            r2_row = model_metrics[model_metrics['metric'].str.contains('R', case=False, na=False)]
            if not r2_row.empty:
                r2_by_model[model] = max(0, r2_row['value'].iloc[0])  # Clip negative R²
        
        # Filter importance data - keep only 'gain' type if available, or 'permutation'
        if 'type' in self.importance_df.columns:
            # Prefer gain, then permutation
            if 'gain' in self.importance_df['type'].values:
                imp_df = self.importance_df[self.importance_df['type'] == 'gain'].copy()
            elif 'permutation' in self.importance_df['type'].values:
                imp_df = self.importance_df[self.importance_df['type'] == 'permutation'].copy()
            else:
                imp_df = self.importance_df.copy()
        else:
            imp_df = self.importance_df.copy()
        
        # Calculate weighted importance
        results = []
        features = imp_df['feature'].unique()
        
        for feature in features:
            feat_data = imp_df[imp_df['feature'] == feature]
            
            # Raw stats
            n_models = len(feat_data)
            raw_importance = feat_data['importance'].values
            
            # Weighted by R²
            weighted_sum = 0
            weight_sum = 0
            
            for _, row in feat_data.iterrows():
                model = row['model_folder']
                imp = row['importance']
                r2 = r2_by_model.get(model, 0)
                
                weighted_sum += imp * r2
                weight_sum += r2
            
            weighted_importance = weighted_sum / weight_sum if weight_sum > 0 else 0
            
            # Rank in each model (1 = most important)
            ranks = []
            for model in feat_data['model_folder'].unique():
                model_imp = imp_df[imp_df['model_folder'] == model].copy()
                model_imp['rank'] = model_imp['importance'].rank(ascending=False)
                feat_rank = model_imp[model_imp['feature'] == feature]['rank'].values
                if len(feat_rank) > 0:
                    ranks.append(feat_rank[0])
            
            avg_rank = np.mean(ranks) if ranks else np.nan
            best_rank = min(ranks) if ranks else np.nan
            
            results.append({
                'feature': feature,
                'n_models': n_models,
                'mean_importance': np.mean(raw_importance),
                'std_importance': np.std(raw_importance),
                'weighted_importance': weighted_importance,
                'avg_rank': avg_rank,
                'best_rank': best_rank,
                # Composite score: weighted importance * (1 / avg_rank)
                'composite_score': weighted_importance * (1 / avg_rank) if avg_rank > 0 else 0
            })
        
        result_df = pd.DataFrame(results)
        result_df = result_df.sort_values('weighted_importance', ascending=False)
        
        return result_df
    
    def get_importance_matrix(self) -> pd.DataFrame:
        """Create a matrix of feature importance by model."""
        if self.importance_df is None:
            return None
        
        # Filter for gain type if available
        if 'type' in self.importance_df.columns:
            if 'gain' in self.importance_df['type'].values:
                imp_df = self.importance_df[self.importance_df['type'] == 'gain']
            else:
                imp_df = self.importance_df[self.importance_df['type'] == self.importance_df['type'].iloc[0]]
        else:
            imp_df = self.importance_df
        
        # Pivot
        matrix = imp_df.pivot_table(
            index='feature',
            columns='model_folder',
            values='importance',
            aggfunc='first'
        )
        
        return matrix


print("MLResultsScanner defined.")

In [ ]:
# CELL 5: Visualization Functions

def plot_model_comparison(comparison_df: pd.DataFrame) -> go.Figure:
    """Create model comparison bar charts."""
    # Identify metric columns
    metric_cols = [c for c in comparison_df.columns if c != 'model_folder']
    n_metrics = len(metric_cols)
    
    # Determine which metrics are "higher is better" vs "lower is better"
    higher_better = ['R²', 'R2', 'Explained Var', 'Explained_Var']
    lower_better = ['RMSE', 'MAE', 'MSE']
    
    fig = make_subplots(
        rows=1, cols=n_metrics,
        subplot_titles=metric_cols
    )
    
    colors = px.colors.qualitative.Set2
    
    for i, metric in enumerate(metric_cols, 1):
        # Sort appropriately
        ascending = any(m in metric for m in lower_better)
        sorted_df = comparison_df.sort_values(metric, ascending=ascending)
        
        # Color by performance
        n = len(sorted_df)
        if ascending:  # Lower is better
            bar_colors = [colors[j % len(colors)] for j in range(n)]
        else:  # Higher is better
            bar_colors = [colors[(n-1-j) % len(colors)] for j in range(n)]
        
        fig.add_trace(
            go.Bar(
                y=sorted_df['model_folder'],
                x=sorted_df[metric],
                orientation='h',
                marker_color=bar_colors,
                text=[f"{v:.4f}" for v in sorted_df[metric]],
                textposition='outside',
                showlegend=False
            ),
            row=1, col=i
        )
        fig.update_xaxes(title_text=metric, row=1, col=i)
    
    fig.update_layout(
        height=400,
        title='Model Performance Comparison (Test Set)',
        showlegend=False
    )
    
    return fig


def plot_importance_heatmap(matrix_df: pd.DataFrame, r2_by_model: Dict = None) -> go.Figure:
    """Create heatmap of feature importance across models."""
    # Sort features by mean importance
    matrix_df = matrix_df.copy()
    matrix_df['mean'] = matrix_df.mean(axis=1)
    matrix_df = matrix_df.sort_values('mean', ascending=True)
    matrix_df = matrix_df.drop('mean', axis=1)
    
    # Sort columns by R² if provided
    if r2_by_model:
        cols_sorted = sorted(matrix_df.columns, key=lambda x: r2_by_model.get(x, 0), reverse=True)
        matrix_df = matrix_df[cols_sorted]
    
    fig = go.Figure(data=go.Heatmap(
        z=matrix_df.values,
        x=matrix_df.columns,
        y=matrix_df.index,
        colorscale='Blues',
        text=[[f"{v:.3f}" if pd.notna(v) else "" for v in row] for row in matrix_df.values],
        texttemplate="%{text}",
        textfont={"size": 10},
        hoverongaps=False
    ))
    
    fig.update_layout(
        title='Feature Importance Across Models',
        xaxis_title='Model',
        yaxis_title='Feature',
        height=max(400, len(matrix_df) * 25)
    )
    
    return fig


def plot_weighted_importance(weighted_df: pd.DataFrame) -> go.Figure:
    """Create weighted importance visualization."""
    # Sort by weighted importance
    df = weighted_df.sort_values('weighted_importance', ascending=True).copy()
    
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=(
            'Weighted Importance (by R²)',
            'Mean Importance (raw)',
            'Composite Score'
        ),
        column_widths=[0.4, 0.3, 0.3]
    )
    
    # Weighted importance
    fig.add_trace(
        go.Bar(
            y=df['feature'],
            x=df['weighted_importance'],
            orientation='h',
            marker_color='steelblue',
            name='Weighted',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # Mean importance with error bars
    fig.add_trace(
        go.Bar(
            y=df['feature'],
            x=df['mean_importance'],
            orientation='h',
            marker_color='coral',
            error_x=dict(type='data', array=df['std_importance']),
            name='Mean ± Std',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Composite score
    df_composite = df.sort_values('composite_score', ascending=True)
    fig.add_trace(
        go.Bar(
            y=df_composite['feature'],
            x=df_composite['composite_score'],
            orientation='h',
            marker_color='purple',
            name='Composite',
            showlegend=False
        ),
        row=1, col=3
    )
    
    fig.update_layout(
        height=max(450, len(df) * 30),
        title='Feature Importance Analysis (weighted by model performance)'
    )
    
    fig.update_xaxes(title_text='Importance × R²', row=1, col=1)
    fig.update_xaxes(title_text='Mean Importance', row=1, col=2)
    fig.update_xaxes(title_text='Weighted × (1/Rank)', row=1, col=3)
    
    return fig


def plot_importance_radar(weighted_df: pd.DataFrame, top_n: int = 8) -> go.Figure:
    """Create radar chart of top features."""
    # Get top N features
    top_features = weighted_df.nlargest(top_n, 'weighted_importance')
    
    # Normalize metrics to 0-1 scale
    metrics = ['weighted_importance', 'mean_importance', 'composite_score']
    
    fig = go.Figure()
    
    for _, row in top_features.iterrows():
        values = []
        for m in metrics:
            max_val = weighted_df[m].max()
            values.append(row[m] / max_val if max_val > 0 else 0)
        values.append(values[0])  # Close the polygon
        
        fig.add_trace(go.Scatterpolar(
            r=values,
            theta=metrics + [metrics[0]],
            fill='toself',
            name=row['feature'],
            opacity=0.6
        ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, 1])
        ),
        showlegend=True,
        title=f'Top {top_n} Features - Multi-metric Comparison',
        height=500
    )
    
    return fig


def plot_r2_vs_importance(scanner: MLResultsScanner) -> go.Figure:
    """Plot feature importance vs model R² to show which features matter in good models."""
    if scanner.importance_df is None or scanner.metrics_df is None:
        return None
    
    # Get R² for each model
    r2_by_model = {}
    for model in scanner.models.keys():
        model_metrics = scanner.metrics_df[
            (scanner.metrics_df['model_folder'] == model) & 
            (scanner.metrics_df['split'] == 'test')
        ]
        r2_row = model_metrics[model_metrics['metric'].str.contains('R', case=False, na=False)]
        if not r2_row.empty:
            r2_by_model[model] = r2_row['value'].iloc[0]
    
    # Filter importance data
    if 'type' in scanner.importance_df.columns:
        if 'gain' in scanner.importance_df['type'].values:
            imp_df = scanner.importance_df[scanner.importance_df['type'] == 'gain'].copy()
        else:
            imp_df = scanner.importance_df.copy()
    else:
        imp_df = scanner.importance_df.copy()
    
    # Add R² to importance data
    imp_df['model_r2'] = imp_df['model_folder'].map(r2_by_model)
    
    # Create scatter plot
    fig = px.scatter(
        imp_df,
        x='model_r2',
        y='importance',
        color='feature',
        hover_data=['model_folder'],
        title='Feature Importance vs Model R²',
        labels={'model_r2': 'Model R² (Test)', 'importance': 'Feature Importance'}
    )
    
    # Add trend lines for top features
    fig.update_traces(marker=dict(size=10, opacity=0.7))
    fig.update_layout(height=500)
    
    return fig


print("Visualization functions defined.")

In [ ]:
# CELL 6: Main Dashboard

# Initialize
scanner = MLResultsScanner()
fb = FolderBrowser('.', label='📁 Select Results Directory')

# Widgets
w_scan_btn = widgets.Button(
    description='🔍 Scan Results',
    button_style='info',
    layout=widgets.Layout(width='150px')
)
w_analyze_btn = widgets.Button(
    description='📊 Analyze & Compare',
    button_style='success',
    layout=widgets.Layout(width='100%', height='45px')
)
w_scan_info = widgets.HTML("<i>Select a folder and click 'Scan Results'</i>")
w_log = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', max_height='200px', overflow='auto'))
w_plots = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', min_height='400px'))

# Event handlers
def on_scan(b):
    with w_log:
        clear_output()
        print(f"Scanning: {fb.path()}")
        
        result = scanner.scan(fb.path())
        
        if result['n_models'] == 0:
            w_scan_info.value = "<span style='color:red'>❌ No ML results found!</span>"
            print("No model folders with *_metrics_*.csv found")
        else:
            w_scan_info.value = f"""
                <div style='background:#e8f6e8; padding:10px; border-radius:5px;'>
                    <b>✅ Found {result['n_models']} models:</b><br>
                    {', '.join(result['models'])}
                </div>
            """
            print(f"Found {result['n_models']} models")
            for m in result['models']:
                print(f"  📁 {m}")

def on_analyze(b):
    if not scanner.models:
        with w_log:
            print("❌ Scan for results first!")
        return
    
    with w_log:
        clear_output()
        print("Loading all results...")
        metrics_df, importance_df = scanner.load_all()
        print(f"Loaded metrics: {len(metrics_df) if metrics_df is not None else 0} rows")
        print(f"Loaded importance: {len(importance_df) if importance_df is not None else 0} rows")
    
    with w_plots:
        clear_output()
        
        # 1. Model Comparison
        print("="*60)
        print("📊 MODEL PERFORMANCE COMPARISON")
        print("="*60)
        
        comparison = scanner.get_model_comparison()
        if comparison is not None:
            display(HTML(comparison.to_html(index=False)))
            
            fig1 = plot_model_comparison(comparison)
            fig1.show()
        
        # 2. Feature Importance Heatmap
        print("\n" + "="*60)
        print("🔥 FEATURE IMPORTANCE HEATMAP")
        print("="*60)
        
        matrix = scanner.get_importance_matrix()
        if matrix is not None:
            # Get R² by model for sorting
            r2_by_model = {}
            for model in scanner.models.keys():
                model_metrics = scanner.metrics_df[
                    (scanner.metrics_df['model_folder'] == model) & 
                    (scanner.metrics_df['split'] == 'test')
                ]
                r2_row = model_metrics[model_metrics['metric'].str.contains('R', case=False, na=False)]
                if not r2_row.empty:
                    r2_by_model[model] = r2_row['value'].iloc[0]
            
            fig2 = plot_importance_heatmap(matrix, r2_by_model)
            fig2.show()
        
        # 3. Weighted Importance
        print("\n" + "="*60)
        print("🏆 WEIGHTED FEATURE IMPORTANCE (by Model R²)")
        print("="*60)
        print("""\nFeatures are weighted by how well the model performed.
A feature that is important in a model with R²=0.9 counts more 
than the same feature in a model with R²=0.3.\n""")
        
        weighted = scanner.get_weighted_importance()
        if weighted is not None:
            # Display table
            display_cols = ['feature', 'n_models', 'weighted_importance', 'mean_importance', 'avg_rank', 'composite_score']
            display_df = weighted[display_cols].copy()
            display_df.columns = ['Feature', '# Models', 'Weighted Imp.', 'Mean Imp.', 'Avg Rank', 'Composite Score']
            display(HTML(display_df.round(4).to_html(index=False)))
            
            fig3 = plot_weighted_importance(weighted)
            fig3.show()
        
        # 4. R² vs Importance scatter
        print("\n" + "="*60)
        print("📈 IMPORTANCE vs MODEL PERFORMANCE")
        print("="*60)
        print("""\nThis plot shows which features are important in the best-performing models.
Points in the upper-right are features that are important in good models.\n""")
        
        fig4 = plot_r2_vs_importance(scanner)
        if fig4:
            fig4.show()
        
        # 5. Summary
        print("\n" + "="*60)
        print("📋 SUMMARY")
        print("="*60)
        
        if comparison is not None:
            r2_col = [c for c in comparison.columns if 'R' in c][0] if any('R' in c for c in comparison.columns) else None
            if r2_col:
                best_model = comparison.loc[comparison[r2_col].idxmax()]
                print(f"\n🥇 Best Model: {best_model['model_folder']} (R² = {best_model[r2_col]:.4f})")
        
        if weighted is not None:
            top_features = weighted.nlargest(5, 'weighted_importance')
            print(f"\n🎯 Top 5 Features (weighted by R²):")
            for i, (_, row) in enumerate(top_features.iterrows(), 1):
                print(f"   {i}. {row['feature']}: {row['weighted_importance']:.4f}")

# Connect events
w_scan_btn.on_click(on_scan)
w_analyze_btn.on_click(on_analyze)

# Build UI
ui = widgets.VBox([
    widgets.HTML("""
        <div style="background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%); 
                    padding: 15px; border-radius: 8px; margin-bottom: 15px;">
            <h2 style="color: white; margin: 0;">🏆 ML Model Comparison Dashboard</h2>
            <p style="color: #e8e8e8; margin: 5px 0 0 0;">Compare models & find the most important features</p>
        </div>
    """),
    
    widgets.HTML("<h3>1️⃣ Select Results Directory</h3>"),
    fb.w,
    widgets.HBox([w_scan_btn]),
    w_scan_info,
    
    widgets.HTML("<hr>"),
    
    widgets.HTML("<h3>2️⃣ Analyze & Compare</h3>"),
    w_analyze_btn,
    
    widgets.HTML("<hr>"),
    
    widgets.HTML("<b>Log:</b>"),
    w_log,
    
    widgets.HTML("<hr>"),
    
    widgets.HTML("<h3>3️⃣ Results</h3>"),
    w_plots
])

display(ui)

---
## 📖 Help

### Metrics Explained:

| Metric | Description |
|--------|-------------|
| **Weighted Importance** | Feature importance × Model R². Features important in better models score higher. |
| **Mean Importance** | Simple average across all models (raw). |
| **Avg Rank** | Average ranking position across models (1 = most important). |
| **Composite Score** | `Weighted Importance × (1/Avg Rank)`. Rewards features that are consistently top-ranked in good models. |

### Interpretation:

- **Weighted Importance**: The main metric. A feature scores high if:
  1. It's important in models that perform well (high R²)
  2. The importance value itself is high

- **Composite Score**: Rewards **consistency**. A feature that ranks #1-2 in all models scores higher than one that ranks #1 in one model but #8 in others.

### How to Use:

1. Navigate to the folder containing your ML results subfolders
2. Click 'Scan Results' to find all models
3. Click 'Analyze & Compare' to generate visualizations

### Expected Directory Structure:

```
results/
├── RandomForest/
│   ├── *_metrics_*.csv
│   ├── *_feature_importance_*.csv
│   └── ...
├── GradientBoosting/
│   └── ...
└── XGBoost/
    └── ...
```